# Rf 5 1 3_Wsfactory_Tools

Organization and simultaneous fits: illustration use of ROOT.RooCustomizer and ROOT.RooSimWSTool interface in factory workspace tool in a complex standalone B physics example




**Author:** Clemens Lange, Wouter Verkerke (C++ version)  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:57 AM.</small></i>

In [1]:
import ROOT


w = ROOT.RooWorkspace("w")

Welcome to JupyROOT 6.18/03


Build a complex example pdf
-----------------------------------------------------------

Make signal model for CPV: A bmixing decay function in t (convoluted with a triple Gaussian resolution model)
times a Gaussian function the reconstructed mass

In [2]:
w.factory(
    "PROD::sig(  BMixDecay::sig_t( dt[-20,20], mixState[mixed=1,unmix=-1], tagFlav[B0=1,B0bar=-1], "
    "tau[1.54], dm[0.472], w[0.05], dw[0], "
    "AddModel::gm({GaussModel(dt,biasC[-10,10],sigmaC[0.1,3],dterr[0.01,0.2]), "
    "GaussModel(dt,0,sigmaT[3,10]), "
    "GaussModel(dt,0,20)},{fracC[0,1],fracT[0,1]}), "
    "DoubleSided ), "
    "Gaussian::sig_m( mes[5.20,5.30], mB0[5.20,5.30], sigmB0[0.01,0.05] ))")

<ROOT.RooProdPdf object ("sig") at 0x563df14e4870>

Make background component: A plain decay function in t times an Argus
function in the reconstructed mass

In [3]:
w.factory("PROD::bkg(  Decay::bkg_t( dt, tau, gm, DoubleSided), "
          "ArgusBG::bkg_m( mes, 5.291, k[-100,-10]))")

<ROOT.RooProdPdf object ("bkg") at 0x563df164e250>

Make composite model from the signal and background component

In [4]:
w.factory("SUM::model( Nsig[5000,0,10000]*sig, NBkg[500,0,10000]*bkg )")

<ROOT.RooAddPdf object ("model") at 0x563df16bcb10>

Example of RooSimWSTool interface
------------------------------------------------------------------

Introduce a flavour tagging category tagCat as observable with 4 states corresponding
to 4 flavour tagging techniques with different performance that require different
parameterizations of the fit model

ROOT.RooSimWSTool operation:
    - Make 4 clones of model (for each tagCat) state, will gain an individual
      copy of parameters w, and biasC. The other parameters remain common
    - Make a simultaneous p.d.f. of the 4 clones assigning each to the appropriate
      state of the tagCat index category

ROOT.RooSimWSTool is interfaced as meta-type SIMCLONE in the factory. The $SplitParam()
argument maps to the SplitParam() named argument in the
ROOT.RooSimWSTool constructor

In [5]:
w.factory(
    "SIMCLONE::model_sim( model, $SplitParam({w,dw,biasC},tagCat[Lep,Kao,NT1,NT2]))")

<ROOT.RooSimultaneous object ("model_sim") at 0x563df18c0740>

Example of RooCustomizer interface
-------------------------------------------------------------------

Class ROOT.RooCustomizer makes clones of existing p.d.f.s with certain prescribed
modifications (branch of leaf node replacements)

Here we take our model (the original before ROOT.RooSimWSTool modifications)
and request that the parameter w (the mistag rate) is replaced with
an expression-based function that calculates w in terms of the Dilution
parameter D that is defined D = 1-2*w

Make a clone model_D of original 'model' replacing 'w' with
'expr('0.5-D/2',D[0,1])'

In [6]:
w.factory("EDIT::model_D(model, w=expr('0.5-D/2',D[0,1]) )")

<ROOT.RooAddPdf object ("model_D") at 0x563df191cae0>

Print workspace contents

In [7]:
w.Print()


RooWorkspace(w) w contents

variables
---------
(D,NBkg,Nsig,biasC,biasC_Kao,biasC_Lep,biasC_NT1,biasC_NT2,dm,dt,dterr,dw,dw_Kao,dw_Lep,dw_NT1,dw_NT2,fracC,fracT,k,mB0,mes,mixState,sigmB0,sigmaC,sigmaT,tagCat,tagFlav,tau,w,w_Kao,w_Lep,w_NT1,w_NT2)

p.d.f.s
-------
RooProdPdf::bkg[ bkg_t * bkg_m ] = 0.307193
RooProdPdf::bkg_Kao[ bkg_t_Kao * bkg_m ] = 0.307193
RooProdPdf::bkg_Lep[ bkg_t_Lep * bkg_m ] = 0.307193
RooProdPdf::bkg_NT1[ bkg_t_NT1 * bkg_m ] = 0.307193
RooProdPdf::bkg_NT2[ bkg_t_NT2 * bkg_m ] = 0.307193
RooArgusBG::bkg_m[ m=mes m0=5.291 c=k p=0.5 ] = 0.279062
RooDecay::bkg_t[ t=dt tau=tau ] = 1.10081
RooDecay::bkg_t_Kao[ t=dt tau=tau ] = 1.10081
RooDecay::bkg_t_Lep[ t=dt tau=tau ] = 1.10081
RooDecay::bkg_t_NT1[ t=dt tau=tau ] = 1.10081
RooDecay::bkg_t_NT2[ t=dt tau=tau ] = 1.10081
RooAddPdf::model[ Nsig * sig + NBkg * bkg ] = 1.88229
RooAddPdf::model_D[ Nsig * sig_model_D + NBkg * bkg ] = 1.5029
RooAddPdf::model_Kao[ Nsig * sig_Kao + NBkg * bkg_Kao ] = 1.88229
RooAddPdf::model

Make workspace visible on command line

In [8]:
ROOT.gDirectory.Add(w)